## Keras Tuner- Decide Number of Hidden Layers And Neuron In Neural Network

In [30]:
!pip install keras-tuner -q

In [31]:
!pip install tensorflow -q

In [59]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt
from keras_tuner.tuners import RandomSearch  ## Decide Number of Hidden Layers And Neuron In Neural Network

In [60]:
df = pd.read_csv('Churn_Modelling.csv')  ## Its classification problem
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [61]:
X = df.iloc[:, 3:13]
y = df.iloc[:, 13]

In [62]:
#Create dummy variables
geography=pd.get_dummies(X["Geography"],drop_first=True)
gender=pd.get_dummies(X['Gender'],drop_first=True)

## Concatenate the Data Frames

X=pd.concat([X,geography,gender],axis=1)

## Drop Unnecessary columns
X=X.drop(['Geography','Gender'],axis=1)

In [63]:
X.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0


## Hyperparameters
1) How many number of hidden layers we should have?

2) How many number of neurons we should have in hidden layers?

3) Learning Rate

In [96]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):                ## How many no's of hidden layer I want
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,       ## Min neuron
                                            max_value=512,      ## MAx neuron
                                            step=32),
                               activation='relu'))              ## activation function for inner layer
    model.add(layers.Dense(1, activation='sigmoid'))            ## activation function for output layer for classification (can use linear for regression)
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',                             ## mean_absolute_error in Regression
        metrics=['accuracy'])                                   ## mean_absolute_error in Regression
    return model

In [97]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',           ## val_mean_absolute_error in Regression
    max_trials=5,
    executions_per_trial=3,             ## combinations model will try is 5*3
    directory='project',                ## store metadata while sere is happening
    project_name='Churn Model')         ## store metadata while sere is happening

Reloading Tuner from project/Churn Model/tuner0.json


In [98]:
tuner.search_space_summary()

Search space summary
Default search space size: 22
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_3 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_4 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_5 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_6

In [99]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [100]:
tuner.search(X_train, y_train,
             epochs=100,
             validation_data=(X_test, y_test))

Trial 6 Complete [00h 05m 07s]
val_accuracy: 0.7933333516120911

Best val_accuracy So Far: 0.7933333516120911
Total elapsed time: 00h 33m 54s


In [101]:
tuner.results_summary()

Results summary
Results in project/Churn Model
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 05 summary
Hyperparameters:
num_layers: 4
units_0: 32
units_1: 288
learning_rate: 0.0001
units_2: 64
units_3: 512
units_4: 384
units_5: 448
units_6: 288
units_7: 384
units_8: 512
units_9: 448
units_10: 352
units_11: 256
units_12: 288
units_13: 480
units_14: 320
units_15: 192
units_16: 96
units_17: 224
units_18: 128
units_19: 160
Score: 0.7933333516120911

Trial 1 summary
Hyperparameters:
num_layers: 15
units_0: 352
units_1: 288
learning_rate: 0.01
units_2: 128
units_3: 448
units_4: 64
units_5: 64
units_6: 224
units_7: 352
units_8: 320
units_9: 192
units_10: 128
units_11: 480
units_12: 128
units_13: 32
units_14: 416
Score: 0.4023333291212718

Trial 0 summary
Hyperparameters:
num_layers: 15
units_0: 384
units_1: 448
learning_rate: 0.0001
units_2: 32
units_3: 32
units_4: 32
units_5: 32
units_6: 32
units_7: 32
units_8: 32
units_9: 32
units_10: 32
units_11: 32
units_1